# Capítulo 02: Instalação e Configuração HTTPFS

Este capítulo cobre a configuração da extensão HTTPFS para acesso HTTP/HTTPS e S3, incluindo a configuração de credenciais e testes de leitura.

In [23]:
import duckdb
import os
import boto3
from botocore.exceptions import ClientError

# ==============================================================================
# SETUP MINIO (SIMULATING S3)
# ==============================================================================
print(f"--- Iniciando Capítulo 02: Instalação e Configuração HTTPFS ---")

MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"
BUCKET_NAME = "learn-duckdb-s3"

# Setup Boto3 client
s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

try:
    s3_client.create_bucket(Bucket=BUCKET_NAME)
    print(f"Bucket '{BUCKET_NAME}' verificado/criado.")
except ClientError:
    print(f"Bucket '{BUCKET_NAME}' já existe ou erro ignorado.")

# Create a sample file
with open("test.csv", "w") as f:
    f.write("col1,col2\n10,20")

try:
    s3_client.upload_file("test.csv", BUCKET_NAME, "test.csv")
    print("Arquivo 'test.csv' enviado com sucesso.")
finally:
    if os.path.exists("test.csv"):
        os.remove("test.csv")

--- Iniciando Capítulo 02: Instalação e Configuração HTTPFS ---
Bucket 'learn-duckdb-s3' já existe ou erro ignorado.
Arquivo 'test.csv' enviado com sucesso.


## ⚙️ Instalação e Carregamento da Extensão HTTPFS

In [24]:
con = duckdb.connect(database=':memory:')

print("\n--- Instalando HTTPFS ---")
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
print("Extensão httpfs carregada.")


--- Instalando HTTPFS ---
Extensão httpfs carregada.


## ✅ Verificação de Extensões

In [25]:
print("\n--- Verificando Extensões Instaladas ---")
extensions = con.execute("SELECT extension_name, installed, loaded FROM duckdb_extensions() WHERE extension_name='httpfs'").df()
print(extensions)


--- Verificando Extensões Instaladas ---
  extension_name  installed  loaded
0         httpfs       True    True


## 🔐 Configuração de Credenciais S3 (MinIO)

Utilizando a sintaxe de `SECRETS` (DuckDB v0.10+).

In [26]:
print("\n--- Configurando Credenciais S3 (MinIO) via SECRET ---")
con.execute(f"""
CREATE OR REPLACE SECRET secret_minio_config (
    TYPE S3,
    KEY_ID '{MINIO_ACCESS_KEY}',
    SECRET '{MINIO_SECRET_KEY}',
    REGION 'us-east-1',
    ENDPOINT '{MINIO_ENDPOINT.replace("http://", "")}',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")
print("Secret configurado com sucesso.")


--- Configurando Credenciais S3 (MinIO) via SECRET ---
Secret configurado com sucesso.


## 🧪 Testando Leitura

In [27]:
print("\n--- Testando Leitura ---")
result = con.execute(f"SELECT * FROM 's3://{BUCKET_NAME}/test.csv'").df()
print(result)


--- Testando Leitura ---
   col1  col2
0    10    20


## 🔄 Teste de Autoload

O DuckDB tenta carregar httpfs automaticamente se detectar s3:// ou https://.

In [28]:
print("\n--- Teste de Autoload (Simulado) ---")
try:
    count = con.execute(f"SELECT count(*) FROM 's3://{BUCKET_NAME}/test.csv'").fetchone()
    print(f"Contagem de linhas: {count[0]}")
except Exception as e:
    print(f"Erro: {e}")


--- Teste de Autoload (Simulado) ---
Contagem de linhas: 1
